In [26]:
import gspread as gs
import pandas as pd

In [27]:
# Get credentials for gspread
gc = gs.service_account(filename="service_account.json")

# Open Google Sheets worksheets
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/12hULH8zk8njDnBph7zEuEtb8nSyKOOGqMTpU45iS8Vg/edit?usp=sharing")

ws1 = sh.worksheet("patientsstatus")
df1 = pd.DataFrame(ws1.get_all_records())

ws2 = sh.worksheet("personaldetails")
df2 = pd.DataFrame(ws2.get_all_records())

# Format data into proper data types
df1["timestamp"] = pd.to_datetime(df1["timestamp"])
df1 = df1.sort_values(by="timestamp", ascending=False)
df1

,timestamp,user_id,default,isdead,length_height,weight,muac,edema,zscore,age,...,dead,isnormal,lheight_rounded,L,M,S,barangay,muni_city,province,region
20,2022-07-26 13:41:26,YvpyvJ,,,90.5,13.4,15.0,no,0.53,2,...,0,1,90.5,-0.3521,12.8392,0.08034,Talipapa,Cabanatuan City,Nueva Ecija,Region III
19,2022-07-26 13:41:24,TSCvbA,,,74.3,10.0,13.0,no,0.87,2,...,0,1,74.0,-0.3521,9.2974,0.08283,Talipapa,Cabanatuan City,Nueva Ecija,Region III
18,2022-07-26 13:41:22,dbHWBt,,,88.2,13.0,10.5,no,0.91,2,...,0,0,88.0,-0.3833,11.972,0.08896,Talipapa,Cabanatuan City,Nueva Ecija,Region III
17,2022-07-26 13:41:20,OpDS71,,,46.7,9.0,14.5,no,10.63,2,...,0,1,46.5,-0.3833,2.7155,0.0904,San Isidro,Cabanatuan City,Nueva Ecija,Region III
16,2022-07-26 13:41:18,tGywJK,,,68.3,8.4,13.0,no,0.4,4,...,0,1,68.0,-0.3521,8.1272,0.08217,San Isidro,Cabanatuan City,Nueva Ecija,Region III
15,2022-07-26 13:41:16,qXQLS6,,,65.0,4.1,14.0,no,-8.05,4,...,0,0,65.0,-0.3521,7.4327,0.08217,San Isidro,Cabanatuan City,Nueva Ecija,Region III
14,2022-07-26 13:41:14,aqnzH7,,,76.1,9.5,11.7,no,0.04,4,...,0,0,76.0,-0.3833,9.4617,0.08983,San Isidro,Cabanatuan City,Nueva Ecija,Region III
13,2022-07-26 13:41:12,qjCALU,,,95.0,15.0,12.2,yes,0.98,0,...,0,0,95.0,-0.3833,13.7146,0.08963,San Isidro,Cabanatuan City,Nueva Ecija,Region III
12,2022-07-26 13:41:10,OwyhLp,,,85.0,14.0,12.0,no,2.39,0,...,0,0,85.0,-0.3833,11.2198,0.08898,San Isidro,Cabanatuan City,Nueva Ecija,Region III
11,2022-07-26 13:41:08,d95a0I,,,76.0,10.0,18.0,yes,0.76,2,...,0,0,76.0,-0.3833,9.3337,0.08992,San Isidro,Cabanatuan City,Nueva Ecija,Region III


In [28]:
# Sort value by most recent to least recent and keep only latest records for each unique child
df1_recent = df1.sort_values(by="timestamp", ascending=False).drop_duplicates( subset=["user_id"], keep="first")

# Summarize SAM, MAM, normal, defaulted cases by day then barangay
df1_recent = df1_recent.groupby([pd.Grouper(key='timestamp', freq='D'), 'barangay', 'muni_city', 'province','region'])[['SAM','MAM','isnormal','defaulted']].sum().reset_index().sort_values('timestamp')

# Store values of currently active SAM and MAM cases
current_active_SAM = df1_recent['SAM'].sum()
current_active_MAM = df1_recent['MAM'].sum()
current_defaulted = df1_recent['defaulted'].sum()

# print("Total active SAM: ", current_active_SAM)
# print("Total active MAM: ", current_active_MAM)
# print("Currently lost to follow ups: ", current_defaulted)
df1_recent

,timestamp,barangay,muni_city,province,region,SAM,MAM,isnormal,defaulted
0,2022-07-26,Bañag,Legazpi City,Albay,Region V,1,0,0,0
1,2022-07-26,Cruzada,Legazpi City,Albay,Region V,1,0,1,0
2,2022-07-26,EM's Barrio,Legazpi City,Albay,Region V,2,2,0,1
3,2022-07-26,San Isidro,Cabanatuan City,Nueva Ecija,Region III,3,2,3,0
4,2022-07-26,Talipapa,Cabanatuan City,Nueva Ecija,Region III,1,0,2,0


In [29]:
# Aggregate all SAM, MAM, default, normal results per barangay
df1_grouped = df1.groupby([pd.Grouper(key='timestamp', freq='D'), 'barangay', 'muni_city', 'province','region'])[['SAM','MAM','defaulted','dead','isnormal']].sum().reset_index().sort_values('timestamp')

df1_grouped

,timestamp,barangay,muni_city,province,region,SAM,MAM,defaulted,dead,isnormal
0,2022-07-05,Bañag,Legazpi City,Albay,Region V,1,0,0,0,0
1,2022-07-26,Bañag,Legazpi City,Albay,Region V,1,0,0,0,0
2,2022-07-26,Cruzada,Legazpi City,Albay,Region V,1,0,0,0,1
3,2022-07-26,EM's Barrio,Legazpi City,Albay,Region V,2,2,1,1,0
4,2022-07-26,San Isidro,Cabanatuan City,Nueva Ecija,Region III,3,2,0,0,3
5,2022-07-26,Talipapa,Cabanatuan City,Nueva Ecija,Region III,1,0,0,0,2


In [30]:
# Get all records of child with user_id of a1 for example
user = "ev29637k"
df1_history = df1.loc[df1["user_id"] == user][['timestamp','length_height','weight','muac','edema','default','zscore','age','status']].sort_values('timestamp', ascending=False)
df1_history

,timestamp,length_height,weight,muac,edema,default,zscore,age,status
1,2022-07-26 13:40:48,69.0,3.0,13.0,no,no,-13.03,2,SEVERE Acute Malnutrition
0,2022-07-05 13:40:48,69.0,1.9,12.42,no,no,-21.02,2,SEVERE Acute Malnutrition


In [33]:
# If you prefer to run the code online instead of on your computer click:
# https://github.com/Coding-with-Adam/Dash-by-Plotly#execute-code-in-browser

from dash import Dash, dcc, html, Output, Input  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import plotly.express as px

# incorporate data into app
df = px.data.medals_long()

# Build your components
app = Dash(__name__, external_stylesheets=[dbc.themes.SOLAR])
mytitle = dcc.Markdown(children='# App that analyzes Olympic medals')

left_jumbotron = dbc.Col(
    html.Div(
        [
            html.H6(["Active SAM: ", current_active_SAM], className="display-6",)
        ],
        className="h-100 p-5 text-white bg-danger rounded-3",
    ),
    md=4,
)

middle_jumbotron = dbc.Col(
    html.Div(
        [
            html.h6("Active SAM", className="display-6",),
            html.H4(current_active_SAM, className="display-4",),
        ],
        className="h-100 p-5 text-white bg-warning rounded-3",
    ),
    md=4,
)

right_jumbotron = dbc.Col(
    html.Div(
        [
            html.H6(["Defaults: ", current_active_SAM], className="display-6",)
        ],
        className="h-100 p-5 text-white bg-dark rounded-3",
    ),
    md=4,
)

jumbotron = dbc.Row(
    [left_jumbotron, middle_jumbotron, right_jumbotron],
    className="align-items-md-stretch",
    style={'margin-bottom': '1em'}
)

mygraph = dcc.Graph(figure={})
dropdown = dcc.Dropdown(options=['Bar Plot', 'Scatter Plot'],
                        value='Bar Plot',  # initial value displayed when page first loads
                        clearable=False)

# Customize your own Layout
app.layout = dbc.Container([mytitle, jumbotron, mygraph, dropdown])

# Callback allows components to interact
@app.callback(
    Output(mygraph, component_property='figure'),
    Input(dropdown, component_property='value')
)
def update_graph(user_input):  # function arguments come from the component property of the Input
    if user_input == 'Bar Plot':
        fig = px.bar(data_frame=df, x="nation", y="count", color="medal")

    elif user_input == 'Scatter Plot':
        fig = px.scatter(data_frame=df, x="count", y="nation", color="medal",
                         symbol="medal")

    return fig  # returned objects are assigned to the component property of the Output


# Run app
if __name__=='__main__':
    app.run_server(port=8050)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Jul/2022 21:12:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2022 21:12:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2022 21:12:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2022 21:12:17] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2022 21:12:17] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2022 21:12:17] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2022 21:12:17] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2022 21:12:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2022 21:12:17] "GET /_favicon.ico?v=2.6.0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2022 21:12:17] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 200 -


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c10d3b81-c7bb-40bf-98a8-a9afcc527b5c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>